# QRBS for dummies


The Quantum Rule-Based System (**QRBS**) NEASQC software library allows their users to implement, transparently, inference of a rule system where the facts are affected by imprecision and the rules have a degree of uncertainty using quantum circuits and eventually a quantum device. 

Classically there are several model implementations for dealing with uncertainty and imprecision. The **QRBS** allows the implementation of the three following models:

1. **Certainty factors**. Proposed by Shortliffe and Buchanan in: *Shortliffe, E. H., & Buchanan, B. G. (1975). A model of inexact reasoning in medicine. Mathematical Biosciences,23(3), 351–379*
2. **Fuzzy logic**: introduced by Loft A. Zadeh in: *Zadeh, L. A. (1965). Fuzzy sets. Information and Control, 8(3), 338–353*
3. **Bayesian networks**: following the work from:  *Borujeni, S. E., Nannapaneni, S., Nguyen, N. H., Behrman, E. C., & Steck, J. E. (2021). Quantum circuit representation of Bayesian networks. Expert Systems with Applications, 176, 114768*

In this notebook we explain how to use **QRBS** to build easy and simple rule systems, implement the associated quantum circuits and execute in a Quantum Process Unit from Eviden myQLM.


In [ ]:
import sys
sys.path.append("../../")

## 1. Facts, rules and islands.

The main ingredients of a rule system are:

* facts: the basic elements of an RBS. They are the smallest unit of information of the RBS.
* rules: relationships between facts (they are like if-then statements in procedural languages). A rule has 2 parts:
    * *left-hand side*: this is the premise of the rule.
    * *right-hand side*: this is the conclusion of the rule
* inference engine: the engine that controls the whole process of applying the complete set of rules of an RBS.

Examples: 
1. The following categorical sentence can be interpreted as an RBS: *If rains I stay at home*. 
    * facts: Fact(rain) and Fact(home).
    * rule: rule: Rule(Fact(rain) is True, Fact(home) is True).
2. The rules can be more sophisticated: *If rains and I am tired I stay at home*:
    * facts: Fact(rain), Fact(tired) and Fact(home).
    * rule: rule: Rule((Fact(rain) is True) and (Fact(tired) is True), Fact(home) is True).    
    
    
The main workflow for defining a **RBS** using **QRBS** are:
1. Instantiate the **QRBS** class
2. Define the facts of the system
3. Define the rules of the system
4. Define the knowledge island of the system.

We are going to develop the first **QRBS** system using the categorical rule: *If it rains I stay at home*

### 1.1 Instantiate the **QRBS** class

For creating a quantum rule-based system using **QRBS** we are going to use the class *QRBS* that allows us to define the complete rule system. This class is inside the module: **neasqc_qrbs.qrbs**

In [ ]:
from neasqc_qrbs.qrbs import QRBS

In [ ]:
qrbs =  QRBS()

### 1.2. Define the facts of the system

Once the **QRBS** is created we need to create the different facts of the system. In the **QRBS** a fact is a Python object that can be created using the **assert_fact** method of the **QRBS** class. To this method, the following inputs should be provided:

* An attribute: a string for identifying the fact
* A value that the attribute takes (it can be a dummy value).
* A precision: **For the moment we are going to think of the precision as a categorical variable** if 0 the fact is false and 1 when the fact is true. By default, the precision is always set to 0.

For the categorical rule: *If rains I stay at home* there are 2 facts: rain and home. 


In [ ]:
#rain fact
rain = qrbs.assert_fact("rain", "rain")
#home fact
home = qrbs.assert_fact("home", "home")

By default both facts will be initialized to a 0 precision (both facts are false)

In [ ]:
print("Precision of rain fact is: ", rain.precision)
print("Precision of home fact is: ", home.precision)

### 1.3. Define the rules of the system

The rules in the **QRBS** are classes that should be created using the method *assert_rule*. A rule has two parts:

* *left-hand side* of a rule is the *if* part that consists of a pattern that matches facts.
* *right hand side* of a rule is the *then* part. This part is set in function of the evaluation of the *left-hand side*.

For the categorical rule: *If rain I stay at home* there are 2 facts: the rule is when *rain*==True then set *home* to True.

In the **QRBS** the *assert_rule* method needs as input the *left-hand side* and *right-hand side* expressions. Additionally when the rule is created one main attribute is the *certainty*. **For the moment we assume that this value is boolean: if 0 the rule does not apply and if 1 the rule does not apply**. By default the *certainty* is fixed to 0. 

In [ ]:
rule = qrbs.assert_rule(rain, home)

In [ ]:
print("The certainty of the rule is: ", rule.certainty)

In [ ]:
# For firing the rule we set certainty should be set to 1
rule.certainty = 1.0

### 1.4. Define the knowledge island of the system.

Finally, we need to group the complete set of rules in *knowledge islands*. For this, we use the method *assert_island* that takes as input a list with the complete rules mandatory. The idea of *knowledge islands* is to chain a set of rules that leads to a final conclusion.  So you can create different *islands* for the same **RBS**.

In [ ]:
island = qrbs.assert_island([rule])

With this we have defined completely the **RBS** system now we need to execute it.

## 2. Builder and Execution of the QRBS

Now we can execute the system to obtain the result for doing this we need to use a *Quantum Process Unit* (**QPU**). This **QPU** will construct the quantum circuit and execute it using a **Eviden myQLM** qpu.

In the **neasqc_qrbs.qrbs** module the **MyQlmQPU** class allows to users  execute the **QRBS** in an easy way using.


In [ ]:
from neasqc_qrbs.qrbs import MyQlmQPU

In [ ]:
my_qlm_qpu = MyQlmQPU()

We are going to fix the *rain* fact to True (1.0) and when the complete **QRBS** is executed the *home* fact should be set to True (1.0)

In [ ]:
# Set the rain fact to 1
print("Precision of rain fact is: ", rain.precision)
rain.precision = 1.0
print("Precision of rain fact is: ", rain.precision)
print("Precision of home fact is: ", home.precision)

In [ ]:
my_qlm_qpu.execute(qrbs)

In [ ]:
print("Precision of home fact is: ", home.precision)

The **MyQlmQPU** only allows the user to use the **PyLinalg** algebra simulator (https://myqlm.github.io/04_api_reference/module_qat/%3Amyqlm%3Amodule_pylinalg/module_simulator.html) and only executes the quantum circuit for 1024 shots. 
 

### 2.1 The select_qpu function and the SelectableQPU class.

In order to give to the user more versatility for executing their **QRBS** systems we have developed the **select_qpu** function and the **SelectableQPU** class:



* **select_qpu** function: under the module **misc.qpu.select_qpu**. This function allows to the user select different **Eviden quantum proccess units (QPUs)** for solving the **QRBS**. 
*  SelectableQPU class: from the module **misc/selectable_qpu**. This class allows to solve a **QRBS** like the **MyQlmQPU** but you can provide a qpu and a different number of shots.

### select_qpu function


The input of this function is a Python dictionary that allows to the user configure easily a **QPU**. The minimum Python dictionary for configuring an ideal **QPU** is presented in the following cell. In this case, the user only has to provide a value to the *qpu_type* key. Depending on the type of simulator desired the following strings should be provided:

* *qlmass_linalg*: to use the **LinAlg Quantum Learning Machine (QLM)** algebra simulator. In this case, the computation will be sent to the **QLM** (https://atos.net/en/solutions/quantum-learning-machine) by using the  Qaptiva QLM as a Service.
* *qlmass_mps*: to use **MPS QLM** simulator. In this case, the computation will be sent to the **QLM** by using the  Qaptiva QLM as a Service.
* *python*: to use the PyLinalg algebra simulator.
* *c*: to use the CLinalg alegbra simulator (https://myqlm.github.io/04_api_reference/module_qat/module_qpus/%3Amyqlm%3Aclinalg.html).
* *linalg*: to use the **LinAlg QLM**. In this case, the user should be inside a **EVIDEN QLM**
* *mps*: to use the **MPS QLM** simulator. In this case, the user should be inside a **EVIDEN QLM**



In [ ]:
# myQLM qpus
from qpu.select_qpu import select_qpu

In [ ]:
# List with the strings that should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]
qpu_config_c = {
    "qpu_type": ideal_qpus[0], 
}
qpu_c = select_qpu(qpu_config_c)
qpu_config_python = {
    "qpu_type": ideal_qpus[1], 
}
qpu_python = select_qpu(qpu_config_python)

#### SelectableQPU class

In [ ]:
from selectable_qpu import SelectableQPU

For example we can use the **CLinalAlg** algebra simulator and 512 shots

In [ ]:
rain.precision = 1.0
home.precision = 0.0
print("Precision of rain fact is: ", rain.precision)
print("Precision of home fact is: ", home.precision)
SelectableQPU.execute(qrbs, qpu = qpu_c, shots = 512)
print("Precision of home fact is: ", home.precision)

For example we can use the **PyLinAlg** algebra simulator and 16 shots

In [ ]:
rain.precision = 1.0
home.precision = 0.0
print("Precision of rain fact is: ", rain.precision)
print("Precision of home fact is: ", home.precision)
SelectableQPU.execute(qrbs, qpu = qpu_python, shots = 16)
print("Precision of home fact is: ", home.precision)

### 2.1 Builder

Inside the **MyQlmQPU** class a **Builder** object is created. The builder is a [Visitor](https://en.wikipedia.org/wiki/Visitor_pattern) that processes a complete *knowledge island* and builds the corresponding quantum circuit associated with the *island* set of rules. 

In the **QRBS** library until three different *builders* were implemented into the **neasqc_qrbs.knowledge_rep** module:

* BuilderImpl: implements **Certainty factors** visitor.
* BuilderFuzzy: implements **Fuzzy logic** visitor.
* BuilderBayes: implements **Bayesian networks** visitor.

The **SelectableQPU** (or **MyQlmQPU**) class builds the different quantum circuits associated with the different islands of **QRBS** using a **Builder** object, executes them and changes the precision of the different facts according to the results of the execution of the quantum circuit.

The user can visualize the associated quantum circuit of an island using the *build* method of the corresponding island and provide it with a desired **Builder** object.

In [ ]:
# The three Builders
from neasqc_qrbs.knowledge_rep import  BuilderImpl, BuilderBayes, BuilderFuzzy

In [ ]:
# Certainty factors engine
qc = island.build(BuilderImpl())
%qatdisplay qc --sv

In [ ]:
# Fuzzy Logic engine
qc = island.build(BuilderFuzzy())
%qatdisplay qc --sv

In [ ]:
# Bayesian Networks engine
qc = island.build(BuilderBayes())
%qatdisplay qc --sv

In fact, we can use different builders when executing the **QRBS** by providing to the execute method of **MyQlmQPU** the following strings:

* *cf*: for using **Certainty factors** engine
* *fuzzy*: for using **Fuzzy Logic** engine
* *bayes*: for using **Bayesian Networks** engine

In [ ]:
rain.precision = 0.0
home.precision = 0.0
print("Precision of rain fact before is: ", rain.precision)
print("Precision of home fact before is: ", home.precision)
SelectableQPU.execute(qrbs,  model='cf', qpu=qpu_c, shots=512)
print("Precision of rain fact after is: ", rain.precision)
print("Precision of home fact after is: ", home.precision)

In [ ]:
rain.precision = 0.0
home.precision = 1.0
print("Precision of rain fact before is: ", rain.precision)
print("Precision of home fact before is: ", home.precision)
SelectableQPU.execute(qrbs,  model='fuzzy', qpu=qpu_python, shots=512)
print("Precision of rain fact after is: ", rain.precision)
print("Precision of home fact after is: ", home.precision)

In [ ]:
rain.precision = 1.0
home.precision = 0.0
print("Precision of rain fact before is: ", rain.precision)
print("Precision of home fact before is: ", home.precision)
SelectableQPU.execute(qrbs,  model='bayes', qpu=qpu_python, shots=512)
print("Precision of rain fact after is: ", rain.precision)
print("Precision of home fact after is: ", home.precision)

## 3. Logical Operators in QRBS

For a **RBS** the rules can have a complex interdependency between facts. For building such interdependencies the operators **AND**, **OR** and **NOT** are used. In the **QRBS** library these three operators are located inside the **neasqc_qrbs.knowledge_rep** module and are implemented as Python classes:

* **AndOperator**
* **OrOperator**
* **NotOperator**

In [ ]:
from neasqc_qrbs.knowledge_rep import AndOperator, OrOperator, NotOperator

We are going to create the **QRBS** associated with the following categorical sentence: *If it rains and I am tired or if it does not rain and I want to read I stay at home*.

In [ ]:
#first: we need to instantiat the QRBS
second_qrbs = QRBS()

#Second define the facts

#rain fact
rain = second_qrbs.assert_fact("rain", "rain")
#tired fact
tired = second_qrbs.assert_fact("tired", "tired")
#read fact
read = second_qrbs.assert_fact("read", "read")
#home fact
home = second_qrbs.assert_fact("home", "home")

# Third Create the Rules:
lrh = OrOperator(AndOperator(rain, tired), AndOperator(NotOperator(rain), read))
rule1 = second_qrbs.assert_rule(lrh, home)
rule1.certainty = 1.0

# Fourht Create the knowledge islands
island = second_qrbs.assert_island([rule1])


We can use the builders for visualizing the circuits

In [ ]:
# Certainty factors engine
qc = island.build(BuilderImpl())
%qatdisplay qc --sv

In [ ]:
# Fuzzy Logic engine
qc = island.build(BuilderFuzzy())
%qatdisplay qc --sv

In [ ]:
# Bayesian Networks engine
qc = island.build(BuilderBayes())
%qatdisplay qc --sv

We can execute now the inference:

In [ ]:
print("Precision of home fact before is: ", home.precision)
SelectableQPU.execute(second_qrbs, model='fuzzy', qpu=qpu_python, shots=16)
print("Precision of home fact after is: ", home.precision)

In [ ]:
rain.precision = 1.0
tired.precision = 1.0
print("Precision of rain fact before is: ", rain.precision)
print("Precision of tired fact before is: ", tired.precision)
print("Precision of home fact before is: ", home.precision)
SelectableQPU.execute(second_qrbs, model='fuzzy', qpu=qpu_python, shots=16)
print("Precision of home fact after is: ", home.precision)

In [ ]:
rain.precision = 0.0
read.precision = 1.0
home.precision = 0.0
print("Precision of rain fact before is: ", rain.precision)
print("Precision of read fact before is: ", read.precision)
print("Precision of home fact before is: ", home.precision)
SelectableQPU.execute(second_qrbs, model='fuzzy', qpu=qpu_c, shots=1024)
print("Precision of home fact after is: ", home.precision)